In [2]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
%matplotlib inline

In [213]:
# from MissingDataScript.py
def fillMissing(inputcsv, outputcsv):
    
    # read input csv - takes time
    df = pd.read_csv(inputcsv, low_memory=False);
    # Fix date bug
    df.cf4fint = ((pd.to_datetime(df.cf4fint) - pd.to_datetime('1960-01-01')) / np.timedelta64(1, 'D')).astype(int);
    
    # drop columns with all NA's (redacted columns)
    df.dropna(axis=1,how='all',inplace=True);
    
    # replace remaining NA's with -10
    df = df.fillna(-10, downcast='infer');
    
    # drop any column that contains a string
    cols_to_remove = []

    for col in df.columns:
        try:
            _ = df[col].astype(float)
        except ValueError:
            #print('Couldn\'t covert %s to float' % col)
            cols_to_remove.append(col)
            pass

    # keep only the columns in df that do not contain string
    df = df[[col for col in df.columns if col not in cols_to_remove]];
    
    # if still NA, replace with 1
    # df = df.fillna(value=1)
    # replace negative values with 1
    #num = df._get_numeric_data()
    #num[num < 0] = 1
    # write filled outputcsv
    df.to_csv(outputcsv, index=False);
    
    return df;
    
# Usage:
# fillMissing('background.csv', 'output.csv')
# filleddf = pd.read_csv('output.csv', low_memory=False)

In [214]:
# read in data
df_raw = fillMissing('fragilefamilieschallenge/background.csv','fragilefamilieschallenge/background_removeNA_and_strings.csv');

In [3]:
df_raw = pd.read_csv('fragilefamilieschallenge/background_removeNA_and_strings.csv',low_memory=False);

In [4]:
# sort data by challengeID
df_sorted = df_raw.iloc[np.argsort(df_raw['challengeID'].values)];

In [5]:
# read in training targets
y_df = pd.read_csv('fragilefamilieschallenge/train.csv',low_memory=False);
# get train ids
train_ids = y_df['challengeID'].values.astype(int);
# test ids are the ones not in train ids
test_ids = np.setdiff1d(np.arange(1,4243),train_ids).astype(int);

In [118]:
from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import Imputer

# remove column for idnum 
df = df_sorted.drop(['idnum'], axis=1);
# index data by challengeID
df.set_index('challengeID',inplace=True);

# remove columns with zero variance
varThresh = VarianceThreshold();
varThresh.fit(df);
df = df.iloc[:,varThresh.get_support()];

# remove columns with other id#s
df = df.drop(df.filter(regex='id\d').axes[1].values,axis=1);

# make new column with count of -9's
df['missing9'] = df[df==-9.0].sum(axis=1)/-9.;

print 'Size before drop majority missing columns: ', df.shape;
nulls = df[df!=-10][df!=-9][df!=-8][df!=-7][df!=-6][df!=-5][df!=-4][df!=-3][df!=-2][df!=-1].isnull().sum().sum();
print 'Number of values to impute: ', nulls, '(', 100.*nulls/(df.shape[0]*df.shape[1]),'%)';

# drop any column that is more than half filled with missing values (not -1 or -2 though)
# remaining missing values converted to NA
#df = df[df>=-2].dropna(axis=1,thresh=2900,inplace=False);
thresh = 2900
#thresh = 3535;
df = df[df!=-10][df!=-9][df!=-8][df!=-7][df!=-6][df!=-5][df!=-4][df!=-3][df!=-2][df!=-1].dropna(axis=1,thresh=thresh,inplace=False);

print 'Size after drop majority missing columns: ', df.shape;

nulls = df.isnull().sum().sum();
print 'Number of values to impute: ', nulls, '(', 100.*nulls/(df.shape[0]*df.shape[1]),'%)'

# impute remaining missing values (now NAs) with most frequent value in column
df = df.apply(lambda x:x.fillna(x.value_counts().index[0],downcast='infer'));
#df = df.fillna(df.mean(), downcast='infer');
# drop any NAs that this may have caused
df = df.dropna(axis=1, how='any',inplace=False);

# do another variance filter
varThresh.fit(df);
df = df.iloc[:,varThresh.get_support()];

# try to distinguish categorical and continuous variables
likely_contin = pd.Series();
for var in df.columns:
    likely_contin[var] = df[var].between(20,99).any() or df[var].dtype==float or df[var].between(1940,2016).all();
likely_cat = ~likely_contin;

print 'Number of likely categorical variables: ', likely_cat.sum(), '(', 100.*likely_cat.sum()/df.shape[1], '%)' 

# one-hot encoding
df = pd.get_dummies(df, prefix=likely_cat[likely_cat==True].axes[0].values, 
                    columns=likely_cat[likely_cat==True].axes[0].values);

# determine variance threshold to get rid of categorical responses with frequency less than 100
dum = np.zeros(4242);
dum[:100]=1;
thresh100 = dum.var();

# a final variance threshold, for low variance 1-hot-encoded columns
# effectively gets rid of low frequency categorical responses
varThresh = VarianceThreshold(threshold=thresh100);
varThresh.fit(df);
df = df.iloc[:,varThresh.get_support()];

# impute -2 and -1 for remaining (continuous) variables with column mean
#df = df[df!=-2][df!=-1].fillna(df.mean());

# separate data into training and testing sets
X_train = df.iloc[train_ids-1];
X_test = df.iloc[test_ids-1];
# index labels by challengeID
y_train = y_df.set_index('challengeID');

# get column names 
feature_names = df.axes[1];
label_names = y_train.axes[1];

# combine training X and y into a single dataframe
train = pd.concat([X_train,y_train],axis=1);

print 'X_train.shape = ', X_train.shape
print 'X_test.shape = ', X_test.shape
print 'y_train.shape = ', y_train.shape
print 'label_names.shape = ', label_names.shape
print 'feature_names.shape = ', feature_names.shape
print 'train.shape = ', train.shape

df.to_csv('imputed_1hot.csv',index=False);
print 'Saving dataframe as imputed_1hot.csv'

Size before drop majority missing columns:  (4242, 10435)
Number of values to impute:  29794879 ( 67.3098322906 %)
Size after drop majority missing columns:  (4242, 1812)
Number of values to impute:  1596008 ( 20.7637698491 %)
Number of likely categorical variables:  1610 ( 89.3451720311 %)
X_train.shape =  (2121, 4467)
X_test.shape =  (2121, 4467)
y_train.shape =  (2121, 6)
label_names.shape =  (6,)
feature_names.shape =  (4467,)
train.shape =  (2121, 4473)
Saving dataframe as imputed_1hot.csv


In [120]:
from sklearn.model_selection import cross_val_score, cross_val_predict, ShuffleSplit
from time import time
def benchmark_all(train_all, X_all, est, label):
    # remove rows that have no training outcomes for this label
    train = train_all.dropna(axis=0, subset=[label]);
    
    # separate data and labels
    X = train.iloc[:,:-6].values;
    y = train[label].values;    

    bootstrap = ShuffleSplit(n_splits=50, test_size=0.1, random_state=0)
    
    score_time = time();
    mse = -cross_val_score(est, X, y, scoring='neg_mean_squared_error',cv=bootstrap, n_jobs=-1);
    r2 = cross_val_score(est, X, y, scoring='r2',cv=bootstrap, n_jobs=-1);  
    print '=============================================================';
    print label;
    print label, 'average mse score = ', mse.mean(), '+/- ', 2.*mse.std();
    print label, 'average r2  score = ', r2.mean(), '+/- ', 2.*r2.std();
    print label, 'bootstrapped scoring time = ', time()-score_time, 's';
    fit_time = time();
    est.fit(X,y);
    predicted = est.predict(X_all);
    print label, 'fit and predict time = ', time()-fit_time, 's';
    
    return mse, r2, predicted

In [121]:
from sklearn.preprocessing import Normalizer
from sklearn.linear_model import RidgeCV
norm = Normalizer(copy=False);
estimator = RidgeCV(alphas=(10.0, 15.0, 20.0), cv=None, fit_intercept=True, normalize=True, scoring='neg_mean_squared_error');
predicted_all = pd.read_csv('fragilefamilieschallenge/prediction.csv');
X_all = df.values;
for label in label_names:
    mse, r2, predicted = benchmark_all(train, X_all, estimator, label);
    print label, 'alpha = ', estimator.alpha_;
    estimator_coefs = estimator.coef_;
    feature_importance = np.argsort(estimator_coefs)[::-1];
    print 'Top 10 features:';
    print feature_names[feature_importance].values[:10];
    feature_values = pd.DataFrame(data=np.sort(estimator_coefs)[::-1],index=feature_names[feature_importance].values,columns=['Weight']);
    feats = feature_values.axes[0].values;
    cont_feats = [s for s in feats if "_" not in s];
    top_cont_feat_idx = np.argwhere(feats==cont_feats[0])
    top_cont_feat = feats[top_cont_feat_idx];
    print 'Top continuous features are'
    for i in range(3):
        top_cont_feat_idx = np.argwhere(feats==cont_feats[i])
        top_cont_feat = feats[top_cont_feat_idx];
        print 'Feature #', top_cont_feat_idx[0][0], ':', top_cont_feat[0][0]
    feature_values.to_csv('results/ridge_feature_rank_lowvar_'+label+'.csv', index=True);
    predicted_all[label] = predicted;

predicted_all.to_csv('results/ridge_impute_1hot_prediction_all.csv',index=False);

gpa
gpa average mse score =  0.379021759622 +/-  0.0821424549111
gpa average r2  score =  0.126525476876 +/-  0.117641859916
gpa bootstrapped scoring time =  80.4229331017 s
gpa fit and predict time =  1.00913310051 s
gpa alpha =  15.0
Top 10 features:
['m1h3_3' 'cm1adult_5' 'm2g13_4' 'cm1ethrace_4' 'm1i1_9' 'f1j7bc_101'
 'm2c3i_2' 'f2h8a3_2' 'm2g1b_102' 'p5i28_3']
Top continuous features are
Feature # 763 : m2b5b
Feature # 1430 : cm2povco
Feature # 1518 : cm4povco
grit
grit average mse score =  0.230280056004 +/-  0.0477060967391
grit average r2  score =  0.0278524705266 +/-  0.0694803861616
grit bootstrapped scoring time =  106.646890879 s
grit fit and predict time =  1.43874692917 s
grit alpha =  20.0
Top 10 features:
['m2c3a_1' 'm1i1_5' 'cf2hhimpb_6' 'cf2hhimp_6' 'cm5mint_1' 'p5q3de_1'
 'm2c3i_3' 'cm4marp_1' 'o5f3_3' 'k5d1e_3']
Top continuous features are
Feature # 1506 : p5h15c
Feature # 1617 : m4k22
Feature # 1713 : m4b3
materialHardship
materialHardship average mse score =  0.02